## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key
import sys, os.path
sys.path.append(os.path.abspath('./'))

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data.csv")
city_data_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,thompson,55.7435,-97.8558,19.29,96,100,5.77,CA,1639574106,overcast clouds
1,1,kodiak,57.7900,-152.4072,20.93,82,52,6.29,US,1639574116,broken clouds
2,2,sao joao da barra,-21.6403,-41.0511,86.09,81,92,15.28,BR,1639574116,overcast clouds
3,3,bilma,18.6853,12.9164,75.43,15,34,11.07,NE,1639574116,scattered clouds
4,4,elbrus,43.2577,42.6443,36.16,85,40,4.72,RU,1639574116,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
2,2,sao joao da barra,-21.6403,-41.0511,86.09,81,92,15.28,BR,1639574116,overcast clouds
21,21,georgetown,5.4112,100.3354,85.95,73,20,6.91,MY,1639574042,few clouds
54,54,beinamar,8.6698,15.3813,96.06,12,95,4.65,TD,1639574131,overcast clouds
59,59,mount isa,-20.7333,139.5000,89.37,43,60,17.27,AU,1639574133,broken clouds
94,94,kolokani,13.5728,-8.0339,92.19,10,100,19.82,ML,1639574143,overcast clouds
105,105,arraial do cabo,-22.9661,-42.0278,87.66,62,0,19.57,BR,1639574147,clear sky
109,109,mocuba,-16.8392,36.9856,96.04,32,99,11.90,MZ,1639574148,overcast clouds
115,115,farim,12.4839,-15.2217,97.18,9,100,4.59,GW,1639574150,overcast clouds
125,125,umm kaddadah,13.6017,26.6876,87.75,17,18,15.50,SD,1639574153,few clouds
139,139,durban,-29.8579,31.0292,91.92,80,15,5.01,ZA,1639573865,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        48
City           48
Lat            48
Lng            48
Max Temp       48
Humidity       48
Cloudiness     48
Wind Speed     48
Country        47
Date           48
Description    48
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()

#dropping any null values
clean_df.dropna(inplace=True)

# checking new data frame
clean_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
2,2,sao joao da barra,-21.6403,-41.0511,86.09,81,92,15.28,BR,1639574116,overcast clouds
21,21,georgetown,5.4112,100.3354,85.95,73,20,6.91,MY,1639574042,few clouds
54,54,beinamar,8.6698,15.3813,96.06,12,95,4.65,TD,1639574131,overcast clouds
59,59,mount isa,-20.7333,139.5000,89.37,43,60,17.27,AU,1639574133,broken clouds
94,94,kolokani,13.5728,-8.0339,92.19,10,100,19.82,ML,1639574143,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp",  "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,sao joao da barra,BR,86.09,overcast clouds,-21.6403,-41.0511,
21,georgetown,MY,85.95,few clouds,5.4112,100.3354,
54,beinamar,TD,96.06,overcast clouds,8.6698,15.3813,
59,mount isa,AU,89.37,broken clouds,-20.7333,139.5000,
94,kolokani,ML,92.19,overcast clouds,13.5728,-8.0339,
105,arraial do cabo,BR,87.66,clear sky,-22.9661,-42.0278,
109,mocuba,MZ,96.04,overcast clouds,-16.8392,36.9856,
115,farim,GW,97.18,overcast clouds,12.4839,-15.2217,
125,umm kaddadah,SD,87.75,few clouds,13.6017,26.6876,
139,durban,ZA,91.92,few clouds,-29.8579,31.0292,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()

clean_hotel_df.dropna(inplace=True)

clean_hotel_df.isnull().sum()


City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))